# OpenAI JSON Mode vs. Function Calling for Data Extraction 

OpenAI just released [JSON Mode](https://platform.openai.com/docs/guides/text-generation/json-mode): This new config constrain the LLM to only generate strings that parse into valid JSON (but no guarantee on validation against any schema).

Before this, the best way to extract structured data from text is via [function calling](https://platform.openai.com/docs/guides/function-calling).  

In this notebook, we explore the tradeoff between the latest [JSON Mode](https://platform.openai.com/docs/guides/text-generation/json-mode) and function calling feature for structured output & extraction.

### Generate synthetic data

We'll start by generating some synthetic data for our data extraction task. Let's ask our LLM for a hypothetical sales transcript.

In [ ]:
from llama_index.llms import OpenAI

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-1106")

In [ ]:
response = llm.complete(
    "Generate a sales call transcript, use real names, talk about a product, discuss some action items"
)

In [ ]:
transcript = response.text
print(transcript)

[Phone rings]

John: Hello, this is John speaking.

Sarah: Hi John, this is Sarah from XYZ Company. I'm calling to discuss our new product, the ABC Widget, and see if it would be a good fit for your business.

John: Hi Sarah, thanks for reaching out. I'm definitely interested in learning more about the ABC Widget. Can you give me a quick overview of what it does?

Sarah: Of course! The ABC Widget is a cutting-edge tool that streamlines your workflow and increases productivity. It's designed to help businesses like yours save time and money by automating repetitive tasks and providing valuable insights into your operations.

John: That sounds really promising. I can see how that could benefit our team. Do you have any case studies or success stories from other companies who have used the ABC Widget?

Sarah: Absolutely, we have several success stories from companies in your industry who have seen significant improvements in their efficiency and bottom line after implementing the ABC Widg

### Setup our desired schema

Let's specify our desired output "shape", as a Pydantic Model.

In [ ]:
from pydantic import BaseModel, Field
from typing import List


class CallSummary(BaseModel):
    """Data model for a call summary."""

    summary: str = Field(
        description="High-level summary of the call transcript. Should not exceed 3 sentences."
    )
    products: List[str] = Field(
        description="List of products discussed in the call"
    )
    rep_name: str = Field(description="Name of the sales rep")
    prospect_name: str = Field(description="Name of the prospect")
    action_items: List[str] = Field(description="List of action items")

### Data extraction with function calling

We can use the `OpenAIPydanticProgram` module in LlamaIndex to make things super easy, simply define a prompt template, and pass in the LLM and pydantic model we've definied.

In [ ]:
from llama_index.program import OpenAIPydanticProgram
from llama_index.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate(
    message_templates=[
        ChatMessage(
            role="system",
            content=(
                "You are an expert assitant for summarizing and extracting insights from sales call transcripts."
            ),
        ),
        ChatMessage(
            role="user",
            content=(
                "Here is the transcript: \n"
                "------\n"
                "{transcript}\n"
                "------"
            ),
        ),
    ]
)
program = OpenAIPydanticProgram.from_defaults(
    output_cls=CallSummary,
    llm=llm,
    prompt=prompt,
    verbose=True,
)

In [ ]:
output = program(transcript=transcript)

Function call: CallSummary with args: {"summary":"Sarah from XYZ Company called John to discuss the new product, the ABC Widget, highlighting its features and benefits. They agreed to schedule a demo for next week and for Sarah to send over case studies and testimonials. John expressed interest in learning more about the product and gathering key stakeholders for the demo.","products":["ABC Widget"],"rep_name":"Sarah","prospect_name":"John","action_items":["Schedule demo for next week","Send over case studies and testimonials"]}


We now have the desired structured data, as a Pydantic Model. 
Quick inspection shows that the results are as we expected. 

In [ ]:
output.dict()

{'summary': 'Sarah from XYZ Company called John to discuss the new product, the ABC Widget, highlighting its features and benefits. They agreed to schedule a demo for next week and for Sarah to send over case studies and testimonials. John expressed interest in learning more about the product and gathering key stakeholders for the demo.',
 'products': ['ABC Widget'],
 'rep_name': 'Sarah',
 'prospect_name': 'John',
 'action_items': ['Schedule demo for next week',
  'Send over case studies and testimonials']}

### Data extraction with JSON mode

Let's try to do the same with JSON mode, instead of function calling

In [ ]:
prompt = ChatPromptTemplate(
    message_templates=[
        ChatMessage(
            role="system",
            content=(
                "You are an expert assitant for summarizing and extracting insights from sales call transcripts.\n"
                "Generate a valid JSON following the given schema below:\n"
                "{json_schema}"
            ),
        ),
        ChatMessage(
            role="user",
            content=(
                "Here is the transcript: \n"
                "------\n"
                "{transcript}\n"
                "------"
            ),
        ),
    ]
)

In [ ]:
messages = prompt.format_messages(
    json_schema=CallSummary.schema_json(), transcript=transcript
)

In [ ]:
output = llm.chat(
    messages, response_format={"type": "json_object"}
).message.content

We get a vaid JSON, but it's only regurgitating the schema we specified, and not actually doing the extraction. 

In [ ]:
print(output)

{
  "title": "CallSummary",
  "description": "Data model for a call summary.",
  "type": "object",
  "properties": {
    "summary": {
      "title": "Summary",
      "description": "High-level summary of the call transcript. Should not exceed 3 sentences.",
      "type": "string"
    },
    "products": {
      "title": "Products",
      "description": "List of products discussed in the call",
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "rep_name": {
      "title": "Rep Name",
      "description": "Name of the sales rep",
      "type": "string"
    },
    "prospect_name": {
      "title": "Prospect Name",
      "description": "Name of the prospect",
      "type": "string"
    },
    "action_items": {
      "title": "Action Items",
      "description": "List of action items",
      "type": "array",
      "items": {
        "type": "string"
      }
    }
  },
  "required": ["summary", "products", "rep_name", "prospect_name", "action_items"]
}


Let's try again by just showing the JSON format we want, instead of specifying the schema

In [ ]:
import json

prompt = ChatPromptTemplate(
    message_templates=[
        ChatMessage(
            role="system",
            content=(
                "You are an expert assitant for summarizing and extracting insights from sales call transcripts.\n"
                "Generate a valid JSON in the following format:\n"
                "{json_example}"
            ),
        ),
        ChatMessage(
            role="user",
            content=(
                "Here is the transcript: \n"
                "------\n"
                "{transcript}\n"
                "------"
            ),
        ),
    ]
)

dict_example = {
    "summary": "High-level summary of the call transcript. Should not exceed 3 sentences.",
    "products": ["product 1", "product 2"],
    "rep_name": "Name of the sales rep",
    "prospect_name": "Name of the prospect",
    "action_items": ["action item 1", "action item 2"],
}

json_example = json.dumps(dict_example)

In [ ]:
messages = prompt.format_messages(
    json_example=json_example, transcript=transcript
)

In [ ]:
output = llm.chat(
    messages, response_format={"type": "json_object"}
).message.content

Now we are able to get the extracted structured data as we expected.

In [ ]:
print(output)

{
  "summary": "During the call, Sarah from XYZ Company introduced the ABC Widget, a tool designed to streamline workflow and increase productivity. John expressed interest in learning more and requested case studies, leading to the decision to schedule a demo for next week.",
  "products": ["ABC Widget"],
  "rep_name": "Sarah",
  "prospect_name": "John",
  "action_items": ["Send case studies and detailed information about the ABC Widget", "Schedule a demo for next week"]
}


### Quick Takeaways

* Function calling remains easier to use for structured data extraction (especially if you have already specified your schema as e.g. a pydantic model)
* While JSON mode enforces the format of the output, it does not help with validation against a specified schema. Directly passing in a schema may not generate expected JSON and may require additional careful formatting and prompting.